In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import time
import re
import os
from tqdm import tqdm
import datetime as dt

In [2]:
# Define the web scraping function
def webcapture(url):
    headers = {
   # pretend I am a browser
   'User-Agent': 'Mozilla/5.0',
   }
    session = requests.Session() #setup session
    data = session.get(url, headers=headers) #scrape the data
    soup = BeautifulSoup(data.text, 'html.parser') #parse the data
    return soup #return the parsed data

In [3]:
# Set up the date range that want to scrape
sdate = dt.date(2020, 1, 20)   # start date
edate = dt.date(2020, 4, 11)   # end date

delta = edate - sdate       # as timedelta
date_range = []
for i in range(delta.days + 1):
    day = sdate + dt.timedelta(days=i)
    date_range.append(str(day))

In [4]:
# Scrape the website
url_base = 'https://trendogate.com/placebydate/2352824/'
usa_trendings = pd.DataFrame()

for idx,date in tqdm(enumerate(date_range)):
    url = url_base+date
    soup = webcapture(url)
    trending_list = [li.get_text() for li in soup.find_all('li','list-group-item')]
    trending = []
    for trend in trending_list:
        trending.append(trend+'='+date)
    usa_trendings = pd.concat([usa_trendings,pd.DataFrame(trending)])
    time.sleep(2)
    
usa_trendings[['trending','date']] = usa_trendings.iloc[:,0].str.split('=',1,expand=True).iloc[:,0:2]

83it [05:02,  2.96s/it]


In [5]:
usa_trendings = usa_trendings.iloc[:,1:3]
# usa_trendings['date'] = usa_trendings['date'].apply(lambda x:'2'+x)

In [6]:
# Save to local 
os.chdir(r'D:\Winter 2020\COVID2019')
usa_trendings.to_csv('usa_trending.csv',index=False)

In [3]:
# Get WOEID for the future scraping
url = 'https://trendogate.com/placebydate/2352824/2020-04-05'

In [23]:
soup = webcapture(url)

In [24]:
woeid = [option.get('value') for option in soup.find_all('option')]
location = [option.get_text() for option in soup.find_all('option')]

In [25]:
# Find cities' WOEID in the USA
usa_id = [re.findall(r'United States',str(loc)) for loc in location]
usa_idx = []
for idx,value in enumerate(usa_id):
    if value != []:
        usa_idx.append(idx)
    else:
        continue


In [26]:
usa_woeid = []
usa_location = []
for idx in usa_idx:
    usa_woeid.append(woeid[idx])
    usa_location.append(location[idx])

In [29]:
# Save to local
usa_df = pd.DataFrame(list(zip(usa_woeid,usa_location)),columns=['woeid','location'])
os.chdir(r'D:\Winter 2020\COVID2019')
usa_df.to_csv('usa_woeid_list.csv',index=False)

In [1]:
### If interested in, feel free to try the following code to scrape cities' trending topics in the USA
# url_base = 'https://trendogate.com/placebydate/'
# usa_trendings = pd.DataFrame()
# for woeid in usa_df['woeid']:
#     for idx,date in tqdm(enumerate(date_range)):
#         url = url_base+woeid+'/'+date
#         soup = webcapture(url)
#         trending_list = [li.get_text() for li in soup.find_all('li','list-group-item')]
#         trending = []
#         for trend in trending_list:
#             trending.append(trend+'='+date+'='+woeid)
#         time.sleep(10)
#     usa_trendings = pd.concat([usa_trendings,pd.DataFrame(trending)])

In [ ]:
# usa_trendings[['trending','date','woeid']] = usa_trendings.iloc[:,0].str.split('=',2,expand=True)

In [ ]:
# usa_trendings = usa_trendings.iloc[:,1:4]

In [ ]:
# usa_trending_topics_full = usa_trendings.merge(usa_df, left_on='woeid', right_on='woeid')

In [ ]:
# os.chdir(r'D:\Winter 2020\COVID2019')
# usa_trending_topics_full.to_csv('usa_trending_topics_full.csv',index=False)